# Wstęp
Zadanie 9 stanowi pierwszy z trzech etapów zajęć poświęconych sieciom rekurencyjnym i predykcji z wykorzystaniem danych multimodalnych. Efektem wszystkich trzech etapów będzie sieć rekurencyjna z warstwą atencji do predykcji kursu kryptowaluty Bitcoin (BTC) w oparciu o dane z giełdy oraz o wyniki analizy emocji komunikatów z mediów społecznościowych, do których również należy utworzyć dedykowany model sieci rekurencyjnej. Plan realizacji etapów wygląda następująco:

1.   EmoTweet - model sieci rekurencyjnej do analizy emocji 
2.   MultiBTC - multimodalny model sieci rekurencyjnej do predykcji kursu BTC
3.   AttEmoTweet & AttMultiBTC - rozszerzenie modeli EmoTweet i MultiBTC o warstwę atencji 

Każdy etap jest traktowany jako oddzielna lista na laboratorium, za którą można otrzymać 10 punktów. 

# Cel ćwiczenia

Celem pierwszego etapu prac jest zapoznanie się z podstawową siecią rekurencyjną LSTM. Ze względu na fakt, że model ten będzie wykorzystany do analizy emocji tekstu, w ramach teorii do zadania zostanie omówiony podstawowy mechanizm konwersji słów w tekście do postaci wektorów dystrybucyjnych (tzw. word embeddings) na podstawie rozwiązania o nazwie `fastText`. Modele będą budowane na ogólnodostępnym zbiorze `TweetEval`, zawierającym podzbiory ręcznie anotowanych tweetów przy pomocy etykiet odnoszących się do następujących zjawisk: 1) emocje (emotion), 2) emotikony (emoji), 3) ironia (irony), 4) mowa nienawiści (hate speech), 5) mowa ofensywna (offensive language), 6) wydźwięk (sentiment), 7) nastawienie (stance). 

# Warunki zaliczenia

Do zaliczenia pierwszego etapu należy utworzyć następujące modele dla min. 2 wybranych zjawisk:

1.   Model bazowy (regresja logistyczna).
2.   Model rekurencyjny oparty o sieć LSTM.

Wytrenowane modele będą wykorzystane w 2 etapie, dlatego proszę je zachować.

# Wektory dystrybucyjne

W przetwarzaniu języka naturalnego, o wektorach dystrybucyjnych (inaczej osadzeniach lub zanurzeniach, ang. word embeddings) mówi się w kontekście reprezentacji słów w tekście, zazwyczaj w postaci wektora liczb rzeczywistych, który koduje znaczenie słowa. Hipoteza dystrybucyjna, u podstawy której leży większość metod reprezentacji, mówi o tym, że słowa, które często współwystępują, mają podobne znaczenie. Wektory dystrybucyjne można uzyskać za pomocą zestawu technik modelowania języka, w których słowa lub frazy są mapowane do wektorów liczb rzeczywistych. Z reguły polega to na matematycznym zanurzeniu z przestrzeni o wielu wymiarach opisujących słowo (konteksty) do ciągłej przestrzeni wektorowej o znacznie mniejszym wymiarze.

Metody generowania tego odwzorowania obejmują sieci neuronowe, redukcję wymiarowości na macierzy współwystępowania słów, modele probabilistyczne lub jawną reprezentację w kontekście, w którym pojawiają się słowa. Wektory dystrybucyjne, używane jako podstawowa reprezentacja wejściowa tekstu, okazały się istotnie poprawiać jakość w wielu zadaniach NLP, takich jak np. rozpoznawanie nazw własnych, określanie części mowy, rozpoznawanie dziedziny tekstu, czy też rozpoznawanie wydźwięku i emocji w tekście. 

# fastText

[fastText](https://fasttext.cc/) jest biblioteką do efektywnego uczenia modeli reprezentacji wektorowych słów oraz do budowania klasyfikatorów tekstu. Modele językowe można budować z wykorzystaniem dwóch popularnych technik: [Continuous Bag of Words](https://www.kdnuggets.com/2018/04/implementing-deep-learning-methods-feature-engineering-text-data-cbow.html) oraz [Skip-Gram](https://towardsdatascience.com/skip-gram-nlp-context-words-prediction-algorithm-5bbf34f84e0c). 

## Instalacja

Pobranie repozytorium projektu:


In [74]:
!git clone https://github.com/facebookresearch/fastText.git

fatal: destination path 'fastText' already exists and is not an empty directory.


Instalacja biblioteki:

In [75]:
!cd fastText && mkdir build && cd build && cmake ..  && make && make install

mkdir: cannot create directory ‘build’: File exists


Instalacja API do Pythona:

In [76]:
!cd fastText && pip install .

Processing /content/fastText
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3086318 sha256=fa0f65d0fcf7521ac505b7ed8621fff2863a5ce2ec81d1a97d12e597f1cb6393
  Stored in directory: /tmp/pip-ephem-wheel-cache-h4yx9p74/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext
  Found existing installation: fasttext 0.9.2
    Uninstalling fasttext-0.9.2:
      Successfully uninstalled fasttext-0.9.2


# Dane do etapu nr 1

## Korpus 
Korpus (zbiór dokumentów) do realizacji etapu nr 1 pochodzą z repozytorium [TweetEval](https://github.com/cardiffnlp/tweeteval). Repozytorium zawiera 7 różnorodnych zbiorów danych, zawierających zanonimizowane wpisy z [Twittera](https://twitter.com), anotowane następującymi zjawiskami: 1) emocje (emotion), 2) emotikony (emoji), 3) ironia (irony), 4) mowa nienawiści (hate speech), 5) mowa ofensywna (offensive language), 6) wydźwięk (sentiment), 7) nastawienie (stance). 

In [77]:
!wget http://jankocon.clarin-pl.eu/share/tweeteval.7z

--2021-05-09 15:38:07--  http://jankocon.clarin-pl.eu/share/tweeteval.7z
Resolving jankocon.clarin-pl.eu (jankocon.clarin-pl.eu)... 156.17.135.34
Connecting to jankocon.clarin-pl.eu (jankocon.clarin-pl.eu)|156.17.135.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17390348 (17M) [application/x-7z-compressed]
Saving to: ‘tweeteval.7z.1’

tweeteval.7z.1      100%[===================>]  16.58M   604 B/s    in 18s     

2021-05-09 15:38:25 (968 KB/s) - ‘tweeteval.7z.1’ saved [17390348/17390348]



In [78]:
!7za x tweeteval.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 17390348 bytes (17 MiB)

Extracting archive: tweeteval.7z
--
Path = tweeteval.7z
Type = 7z
Physical Size = 17390348
Headers Size = 1810
Method = LZMA2:24
Solid = +
Blocks = 1

  0%    
Would you like to replace the existing file:
  Path:     ./tweeteval/.git/HEAD
  Size:     21 bytes (1 KiB)
  Modified: 2021-04-09 11:24:03
with the file from archive:
  Path:     tweeteval/.git/HEAD
  Size:     21 bytes (1 KiB)
  Modified: 2021-04-09 11:24:03
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? y

 10% 33 - tweeteval/.git/HEAD                             
Would you like to replace the existing file:
  Path:     ./tweeteval/.git/conf

## Zawartość korpusu

W katalogu głównym (tweeteval) znajdują się następujące elementy:
*   `datasets` - katalog ze zbiorami danych
   * `emotion` - tweety anotowane emocjami 
     * `mapping.txt` - identyfikatory etykiet oraz ich opis
     * `train_text.txt` - wpisy z Twittera (część ucząca)
     * `train_labels.txt` - etykiety wpisów z Twittera (część ucząca)
     * `test_*.txt, valid_*.txt` - j.w. (część testowa i walidacyjna)
   * `emoji` - tweety anotowane emotikonami
   * `...` - katalogi zawierające tweety anotowane pozostałymi zjawiskami
*   `predictions` - katalog z przykładowymi predykcjami
   * `emotion.txt` - etykiety modelu predykcyjnego dla części testowej danych `emotion`
   * `emoji.txt` - j.w. dla cz. testowej danych `emoji`
   * `...` - j.w. dla pozostałych danych
*   `evaluation_script.py` - skrypt do ewaluacji 

## Model języka

Na potrzeby zadania został przygotowany model Skip-Gram reprezentacji wektorowej słów, zbudowany na wielkim korpusie tweetów dotyczących kursu BTC. Wersja binarna tego modelu dostępna jest w 2 wariantach:
* [wektory 100-elementowe (1.7GB)](http://jankocon.clarin-pl.eu/share/fasttext_tweetmodel_btc_sg_100_en.bin)
* [wektory 20-elementowe (350MB)](http://jankocon.clarin-pl.eu/share/fasttext_tweetmodel_btc_sg_20_en.bin)

Na potrzeby prezentacji przykładowego rozwiązania zostanie wykorzystany mniejszy model. Do realizacji ostatecznego rozwiązania należy wykorzystać większy model. 





# Model bazowy rozpoznawania emocji

Model bazowy, zbudowany z wykorzystaniem narzędzia fastText (oparty o regresję logistyczną), będzie punktem wyjścia do porównania się z modelami opartymi o sieci LSTM, których skonstruowanie i ewaluacja na wybranych zadaniach będzie celem etapu nr 1. 

Pobranie mniejszego modelu reprezentacji języka tweetów:


In [79]:
!wget http://jankocon.clarin-pl.eu/share/fasttext_tweetmodel_btc_sg_20_en.bin

--2021-05-09 15:59:23--  http://jankocon.clarin-pl.eu/share/fasttext_tweetmodel_btc_sg_20_en.bin
Resolving jankocon.clarin-pl.eu (jankocon.clarin-pl.eu)... 156.17.135.34
Connecting to jankocon.clarin-pl.eu (jankocon.clarin-pl.eu)|156.17.135.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 365240858 (348M) [application/octet-stream]
Saving to: ‘fasttext_tweetmodel_btc_sg_20_en.bin.1’

fasttext_tweetmodel 100%[===================>] 348.32M  10.9MB/s    in 34s     

2021-05-09 15:59:57 (10.2 MB/s) - ‘fasttext_tweetmodel_btc_sg_20_en.bin.1’ saved [365240858/365240858]



Wydobycie słownika wektorów z binarnego modelu języka:

In [80]:
!python fastText/python/doc/examples/bin_to_vec.py fasttext_tweetmodel_btc_sg_20_en.bin > fasttext_tweetmodel_btc_sg_20_en.vec

Dodanie prefiksu `__label__` do etykiet zbioru `emotion`:

In [81]:
!sed 's/^/__label__/g' tweeteval/datasets/emotion/train_labels.txt > train_labels_emo.txt
!sed 's/^/__label__/g' tweeteval/datasets/emotion/test_labels.txt > test_labels_emo.txt
!sed 's/^/__label__/g' tweeteval/datasets/emotion/val_labels.txt > val_labels_emo.txt

Przygotowanie zbioru uczącego, testowego i walidacyjnego w formacie `fastText`:

In [82]:
!paste -d " " tweeteval/datasets/emotion/train_text.txt train_labels_emo.txt > train_emo.txt
!paste -d " " tweeteval/datasets/emotion/test_text.txt test_labels_emo.txt > test_emo.txt
!paste -d " " tweeteval/datasets/emotion/val_text.txt val_labels_emo.txt > val_emo.txt

Trenowanie modelu z wykorzystaniem wejścia `train_emo.txt`, z określeniem wyjściowej nazwy modelu `emo_model`, dla wektorów słów o wymiarze `20`, z wykorzystaniem pretrenowanych wektorów z pliku `fasttext_tweetmodel_btc_sg_20_en.vec` i z uruchomieniem dostrajania hiperparametrów na zbiorze walidacyjnym `val_emo.txt`:

In [83]:
!fasttext supervised -input train_emo.txt -output emo_model -dim 20 -pretrainedVectors fasttext_tweetmodel_btc_sg_20_en.vec -autotune-validation val_emo.txt 

Progress: 100.0% Trials:   22 Best score:  0.689840 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  12887
Number of labels: 4
Progress: 100.0% words/sec/thread:   77277 lr:  0.000000 avg.loss:  0.529891 ETA:   0h 0m 0s


Podstawowa ewaluacja modelu z wykorzystaniem `fastText`, wynikiem jest precyzja (P - precision) i kompletność (R - recall) w wariancie [weighted](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html).

In [84]:
!fasttext test emo_model.bin test_emo.txt

N	1421
P@1	0.69
R@1	0.69


Rozszerzona ewaluacja modelu z wykorzystaniem `fastText`, wynikiem jest precyzja (P - precision), kompletność (R - recall) oraz F1-score dla każdej etykiety w wariancie [weighted](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html).

In [85]:
!fasttext test-label emo_model.bin test_emo.txt

F1-Score : 0.760887  Precision : 0.702580  Recall : 0.829749   __label__0
F1-Score : 0.677043  Precision : 0.670951  Recall : 0.683246   __label__3
F1-Score : 0.661538  Precision : 0.736301  Recall : 0.600559   __label__1
F1-Score : 0.411765  Precision : 0.518519  Recall : 0.341463   __label__2
N	1421
P@1	0.690
R@1	0.690


Przygotowanie danych do ewaluacji z wykorzystaniem skryptu dołączonego do zbioru TweetEval:

In [86]:
!mkdir predictions2

mkdir: cannot create directory ‘predictions2’: File exists


In [87]:
!fasttext predict emo_model.bin tweeteval/datasets/emotion/test_text.txt | sed 's/__label__//g' > predictions2/emotion.txt

Uruchomienie ewaluacji. Oprócz wyników P, R, F1 [weighted]((https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)) dla każdej etykiety, otrzymujemy również wyniki w wariancie [macro]((https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)). **Ostateczną miarą (TweetEval Score) jest miara F1-score w wariancie macro i tę miarę proszę traktować jako kluczową przy porównywaniu rozwiązań.**

In [88]:
!python tweeteval/evaluation_script.py --tweeteval_path tweeteval/datasets --predictions_path predictions2 --task emotion

0 {'precision': 0.7025796661608498, 'recall': 0.8297491039426523, 'f1-score': 0.76088742810189, 'support': 558}
1 {'precision': 0.7363013698630136, 'recall': 0.6005586592178771, 'f1-score': 0.6615384615384615, 'support': 358}
2 {'precision': 0.5185185185185185, 'recall': 0.34146341463414637, 'f1-score': 0.411764705882353, 'support': 123}
3 {'precision': 0.6709511568123393, 'recall': 0.6832460732984293, 'f1-score': 0.6770428015564202, 'support': 382}
accuracy 0.6903589021815623
macro avg {'precision': 0.6570876778386803, 'recall': 0.6137543127732763, 'f1-score': 0.6278083492697812, 'support': 1421}
weighted avg {'precision': 0.6866407204847322, 'recall': 0.6903589021815623, 'f1-score': 0.6830987777126711, 'support': 1421}
------------------------------
TweetEval Score (emotion): 0.6278083492697812


# Budowa modeli EmoTweet

W tej sekcji Państwa zadaniem będzie przygotowanie modeli sieci LSTM oraz modeli bazowych opartych o regresję logistyczną (fastText) dla wybranych 2 zjawisk ze zbioru TweetEval. Dla sieci LSTM kolejne jednostki sieci rekurencyjnej na wejściu dostają reprezentację wektorową kolejnych wyrazów w tekście. Wyjście z ostatniej jednostki podlega klasyfikacji. W celu usprawnienia zadania, przedstawiona zostanie metoda reprezentacji wektorowej tekstu z wykorzystaniem Pythonowego API do narzędzia fastText. Do ewaluacji modeli należy wykorzystać uprzednio zaprezentowany skrypt `tweeteval/evaluation_script.py`.

## Wektoryzacja tekstu


In [2]:
# inicjalizacja biblioteki
import fasttext

In [3]:
# ładowanie modelu
MODEL_PATH = 'fasttext_tweetmodel_btc_sg_20_en.bin'
model = fasttext.load_model(MODEL_PATH)

In [4]:
# wczytanie danych treningowych
import pandas as pd
TRAIN_PATH = 'tweeteval/datasets/emotion/train_text.txt'
train_texts = pd.read_csv(TRAIN_PATH, sep='\t', header=None)
train_texts

,0
0,“Worry is a down payment on a problem you may ...
1,My roommate: it's okay that we can't spell bec...
2,No but that's so cute. Atsu was probably shy a...
3,Rooneys fucking untouchable isn't he? Been fuc...
4,it's pretty depressing when u hit pan on ur fa...
...,...
3252,I get discouraged because I try for 5 fucking ...
3253,The @user are in contention and hosting @user ...
3254,@user @user @user @user @user as a fellow UP g...
3255,You have a #problem? Yes! Can you do #somethin...


In [5]:
# wektoryzacja pierwszego tekstu
first_text = train_texts[0][0]
for word in fasttext.tokenize(first_text):
  print(word, model.get_word_id(word), model.get_word_vector(word))

“Worry -1 [-0.04189867  0.15429688  0.96717507  1.3809655   0.49123076 -0.5447607
 -0.11276884  0.20356484 -1.0640966  -1.6616327   0.03930127 -0.7224096
  0.21334486 -0.5872285   0.2898182   0.81751084 -1.6077403   1.8038087
  0.4850348   1.0643197 ]
is 6 [ 0.24099417  0.13544752  0.7251924   0.32544732  0.27421224  0.31903243
  0.7501186   0.22853182 -0.91543657  0.08587569  0.13866538 -0.38624704
 -0.30637258  0.13666666 -0.43992838 -0.12443608 -1.0383893  -0.06567164
  0.17007533 -0.16708991]
a 7 [-0.00810981 -0.03934941  0.81658655  0.56301105  0.43812367  0.29547286
  0.4691784   0.07483605 -0.58705056  0.28240088 -0.6339584  -0.16187707
 -0.23376046 -0.1245347   0.03071329 -0.07603034 -0.9066614  -0.07007706
  0.4522892  -0.15033531]
down 174 [ 0.9175071  -1.0815151   0.07119758  0.34226617  0.9607946   0.5973182
  0.91058624 -0.32068744 -0.72137564  1.2241784  -0.1882128  -0.23591968
 -0.02596712 -0.10194965 -0.09553405  0.36303622  0.22354192  0.4901933
  0.5405883   0.5965071

Proszę zwrócić uwagę, że fastText jest w stanie przyporządkować reprezentację wektorową nawet dla takich słów, których model języka nie widział w trakcie uczenia (pierwszy token wejściowego tekstu). 

## Model klasyfikacji tekstu LSTM

In [6]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

batch_size = 256

def read_to_dataloader(dataset_path, batch_size, train_test_or_val = 'train'):
    assert train_test_or_val in ['train', 'test', 'val']
    X = pd.read_csv(os.path.join(dataset_path, f'{train_test_or_val}_text.txt'), sep=' \n', header=None)
    y = pd.read_csv(os.path.join(dataset_path, f'{train_test_or_val}_labels.txt'), sep=' \n', header=None)

    X_tensor = nn.utils.rnn.pad_sequence([torch.Tensor([model.get_word_vector(word) for word in fasttext.tokenize(tweet)]) for tweet in X[0]], batch_first=True)
    y_tensor = torch.Tensor(y.values).long() 

    return DataLoader(TensorDataset(X_tensor, y_tensor), batch_size=batch_size)

In [7]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.5):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.dense = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.dense(x[:, -1])
        return x


## Trenowanie modeli LSTM dla ZJAWISKO_1 i ZJAWISKO_2

In [8]:
import torch.optim as optim

def fit(
    model: nn.Module, 
    optimiser: optim.Optimizer, 
    loss_fn: torch.nn.CrossEntropyLoss, 
    train_dl: DataLoader, 
    val_dl: DataLoader, 
    epochs: int, 
    print_metrics: str = True
):
  for epoch in range(epochs):
      model.train()
      for X_batch, y_batch in train_dl:
          y_pred = model(X_batch.cuda())
          loss = loss_fn(y_pred, y_batch.squeeze().cuda())
          loss.backward()
          optimiser.step()
          optimiser.zero_grad()
      if print_metrics:
          losses_sum = 0.0 
          model.eval()
          with torch.no_grad():
              for X_batch, y_batch in val_dl: 
                  y_pred = model(X_batch.cuda())
                  losses_sum+=loss_fn(y_pred, y_batch.squeeze().cuda())
              print(
                  f"Epoch {epoch}: "
                  f"train loss = {loss:.3f} "
                  f"validation loss = {losses_sum/len(val_dl):.3f}"
              )

In [23]:
datasets_folder_path = os.path.join('tweeteval', 'datasets')
sentiment_train_dl = read_to_dataloader(os.path.join(datasets_folder_path, 'sentiment'), batch_size, 'train')
sentiment_test_dl = read_to_dataloader(os.path.join(datasets_folder_path, 'sentiment'), batch_size, 'test')

output_size = 3
hidden_size = 128
num_layers = 4
lr=0.001
epochs = 50

sentiment_model = LSTMModel(20, hidden_size, num_layers, output_size).cuda()
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(sentiment_model.parameters(), lr=lr)

fit(
    sentiment_model, 
    optim,
    loss_fn, 
    sentiment_train_dl,
    sentiment_test_dl,
    epochs, 
    print_metrics = True
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


Epoch 0: train loss = 0.954 validation loss = 0.943
Epoch 1: train loss = 0.961 validation loss = 0.946
Epoch 2: train loss = 0.927 validation loss = 0.897
Epoch 3: train loss = 0.900 validation loss = 0.875
Epoch 4: train loss = 0.879 validation loss = 0.868
Epoch 5: train loss = 0.872 validation loss = 0.852
Epoch 6: train loss = 0.852 validation loss = 0.877
Epoch 7: train loss = 0.850 validation loss = 0.880
Epoch 8: train loss = 0.817 validation loss = 0.863


KeyboardInterrupt: ignored

In [21]:
emoji_train_dl = read_to_dataloader(os.path.join(datasets_folder_path, 'emoji'), batch_size, 'train')
emoji_test_dl = read_to_dataloader(os.path.join(datasets_folder_path, 'emoji'), batch_size, 'test')

output_size = 20
hidden_size = 32
num_layers = 4
lr=0.001
epochs = 50

emoji_model = LSTMModel(20, hidden_size, num_layers, output_size).cuda()
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(emoji_model.parameters(), lr=lr)

fit(
    emoji_model, 
    optim,
    loss_fn, 
    emoji_train_dl,
    emoji_test_dl,
    epochs, 
    print_metrics = True
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


Epoch 0: train loss = 2.821 validation loss = 2.756
Epoch 1: train loss = 2.819 validation loss = 2.755
Epoch 2: train loss = 2.821 validation loss = 2.755
Epoch 3: train loss = 2.604 validation loss = 2.499
Epoch 4: train loss = 2.496 validation loss = 2.446
Epoch 5: train loss = 2.482 validation loss = 2.401
Epoch 6: train loss = 2.478 validation loss = 2.387
Epoch 7: train loss = 2.475 validation loss = 2.376
Epoch 8: train loss = 2.462 validation loss = 2.373
Epoch 9: train loss = 2.466 validation loss = 2.375


KeyboardInterrupt: ignored

## Trenowanie modeli LR (fastText) dla ZJAWISKO_1 i ZJAWISKO_2

In [13]:
!sed 's/^/__label__/g' tweeteval/datasets/sentiment/train_labels.txt > train_labels_sen.txt
!sed 's/^/__label__/g' tweeteval/datasets/sentiment/test_labels.txt > test_labels_sen.txt
!sed 's/^/__label__/g' tweeteval/datasets/sentiment/val_labels.txt > val_labels_sen.txt

!paste -d " " tweeteval/datasets/sentiment/train_text.txt train_labels_sen.txt > train_sen.txt
!paste -d " " tweeteval/datasets/sentiment/test_text.txt test_labels_sen.txt > test_sen.txt
!paste -d " " tweeteval/datasets/sentiment/val_text.txt val_labels_sen.txt > val_sen.txt

In [14]:
!sed 's/^/__label__/g' tweeteval/datasets/emoji/train_labels.txt > train_labels_emoji.txt
!sed 's/^/__label__/g' tweeteval/datasets/emoji/test_labels.txt > test_labels_emoji.txt
!sed 's/^/__label__/g' tweeteval/datasets/emoji/val_labels.txt > val_labels_emoji.txt

!paste -d " " tweeteval/datasets/emoji/train_text.txt train_labels_emoji.txt > train_emoji.txt
!paste -d " " tweeteval/datasets/emoji/test_text.txt test_labels_emoji.txt > test_emoji.txt
!paste -d " " tweeteval/datasets/emoji/val_text.txt val_labels_emoji.txt > val_emoji.txt

In [15]:
!fasttext supervised -input train_sen.txt -output sen_model -dim 20 -pretrainedVectors fasttext_tweetmodel_btc_sg_20_en.vec -autotune-validation val_sen.txt 

Progress: 100.0% Trials:   14 Best score:  0.696500 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  106361
Number of labels: 3
Progress: 100.0% words/sec/thread:   66940 lr:  0.000000 avg.loss:  0.589642 ETA:   0h 0m 0s


In [16]:
!fasttext supervised -input train_emoji.txt -output emoji_model -dim 20 -pretrainedVectors fasttext_tweetmodel_btc_sg_20_en.vec -autotune-validation val_emoji.txt 

Progress: 100.0% Trials:   17 Best score:  0.254000 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  94918
Number of labels: 20
Progress: 100.0% words/sec/thread:  267616 lr:  0.000000 avg.loss:  1.891987 ETA:   0h 0m 0s


## Ewaluacja modeli na danych testowych dla zjawiska ZJAWISKO_1

In [29]:
if not os.path.exists('predictions_LSTM'):
  os.mkdir('predictions_LSTM')

In [30]:
sentiment_model.eval()
y_pred_all = []
for t_inp, _ in sentiment_test_dl:
    t_inp = t_inp.cuda()
    for x in sentiment_model(t_inp):
        y_pred_all.append(torch.argmax(x).cpu().numpy().tolist())

np.savetxt('predictions_LSTM/sentiment.txt', y_pred_all, fmt="%i")

In [32]:
!python tweeteval/evaluation_script.py --tweeteval_path tweeteval/datasets --predictions_path predictions_LSTM --task sentiment

0 {'precision': 0.7607305936073059, 'recall': 0.20971802618328297, 'f1-score': 0.32879415827906056, 'support': 3972}
1 {'precision': 0.5642701525054467, 'recall': 0.8288697995620684, 'f1-score': 0.6714422158548232, 'support': 5937}
2 {'precision': 0.5405186385737439, 'recall': 0.5616842105263158, 'f1-score': 0.5508982035928144, 'support': 2375}
accuracy 0.5770107456854445
macro avg {'precision': 0.6218397948954989, 'recall': 0.5334240120905557, 'f1-score': 0.5170448592422328, 'support': 12284}
weighted avg {'precision': 0.6232029941261558, 'recall': 0.5770107456854445, 'f1-score': 0.537341750712101, 'support': 12284}
------------------------------
TweetEval Score (sentiment): 0.5334240120905557


In [33]:
!fasttext predict sen_model.bin tweeteval/datasets/sentiment/test_text.txt | sed 's/__label__//g' > predictions2/sentiment.txt

In [34]:
!python tweeteval/evaluation_script.py --tweeteval_path tweeteval/datasets --predictions_path predictions2 --task sentiment

0 {'precision': 0.6505515898767035, 'recall': 0.5047834843907352, 'f1-score': 0.5684717890558549, 'support': 3972}
1 {'precision': 0.6256402297066583, 'recall': 0.6789624389422267, 'f1-score': 0.6512116316639741, 'support': 5937}
2 {'precision': 0.5498368974266038, 'recall': 0.6387368421052632, 'f1-score': 0.5909622126996493, 'support': 2375}
accuracy 0.6148648648648649
macro avg {'precision': 0.6086762390033219, 'recall': 0.607494255146075, 'f1-score': 0.6035485444731594, 'support': 12284}
weighted avg {'precision': 0.6190393674818366, 'recall': 0.6148648648648649, 'f1-score': 0.6128092362813853, 'support': 12284}
------------------------------
TweetEval Score (sentiment): 0.607494255146075


## Ewaluacja modeli na danych testowych dla zjawiska ZJAWISKO_2

In [35]:
emoji_model.eval()
y_pred_all = []
for t_inp, _ in emoji_test_dl:
    t_inp = t_inp.cuda()
    for x in emoji_model(t_inp):
        y_pred_all.append(torch.argmax(x).cpu().numpy().tolist())

np.savetxt('predictions_LSTM/emoji.txt', y_pred_all, fmt="%i")

In [36]:
!python tweeteval/evaluation_script.py --tweeteval_path tweeteval/datasets --predictions_path predictions_LSTM --task emoji

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
0 {'precision': 0.7766925848663, 'recall': 0.7585664011854047, 'f1-score': 0.7675224887556221, 'support': 10798}
1 {'precision': 0.1246473840598789, 'recall': 0.686128364389234, 'f1-score': 0.21096858388770415, 'support': 4830}
10 {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1432}
11 {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1949}
12 {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1265}
13 {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1114}
14 {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1306}
15 {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1244}
16 {'precision': 0.0, 'recall': 0.0

In [37]:
!fasttext predict emoji_model.bin tweeteval/datasets/emoji/test_text.txt | sed 's/__label__//g' > predictions2/emoji.txt

In [38]:
!python tweeteval/evaluation_script.py --tweeteval_path tweeteval/datasets --predictions_path predictions2 --task emoji

0 {'precision': 0.726998491704374, 'recall': 0.7588442304130395, 'f1-score': 0.7425800897186098, 'support': 10798}
1 {'precision': 0.22086314331871681, 'recall': 0.39627329192546584, 'f1-score': 0.2836395969176052, 'support': 4830}
10 {'precision': 0.2691194708557255, 'recall': 0.454608938547486, 'f1-score': 0.3380940015580369, 'support': 1432}
11 {'precision': 0.41211401425178146, 'recall': 0.3560800410466906, 'f1-score': 0.38205339939443983, 'support': 1949}
12 {'precision': 0.4825090470446321, 'recall': 0.31620553359683795, 'f1-score': 0.3820439350525311, 'support': 1265}
13 {'precision': 0.034482758620689655, 'recall': 0.0026929982046678637, 'f1-score': 0.004995836802664447, 'support': 1114}
14 {'precision': 0.09671179883945841, 'recall': 0.03828483920367534, 'f1-score': 0.0548546352166758, 'support': 1306}
15 {'precision': 0.15011547344110854, 'recall': 0.0522508038585209, 'f1-score': 0.07751937984496124, 'support': 1244}
16 {'precision': 0.06208053691275168, 'recall': 0.032090199